In [1]:
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D,GlobalAveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
import scipy.misc
from datetime import datetime
from matplotlib.pyplot import imshow
%matplotlib inline

# import keras.backend as K
# K.set_image_data_format('channels_last')
# K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [3]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [4]:
def ResNet50(input_shape=(64, 64, 3), classes=6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    #X = AveragePooling2D((2,2), name="avg_pool")(X)
    X = GlobalAveragePooling2D(name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    #X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [5]:
model = ResNet50(input_shape = (256, 256, 3), classes = 11)
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
# # standard model load from Keras API
# from tensorflow.keras.applications import ResNet50
# base_model = ResNet50(include_top=True, weights=None)
# base_model.summary()


In [ ]:
# w = base_model.layers[10].get_weights()[0]
# w

In [ ]:
# x=base_model.layers[-2].output
# predictions = Dense(11,activation='softmax')(x)
# model = Model(inputs=base_model.input,outputs=predictions)
# model.summary()

In [ ]:
# # standard model load from Keras API
# from tensorflow.keras.applications import ResNet50
# base_model = ResNet50(include_top=False, input_shape= (256,256,3),weights=None)
# base_model.summary()


In [ ]:
# # add a global spatial average pooling layer
# x = base_model.output
# x = GlobalAveragePooling2D(name="avg_pool")(x)

# predictions = Dense(11, activation='softmax')(x)

In [ ]:
# # this is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)
# model.summary()

In [ ]:
# # standard model load from Keras API
# from tensorflow.keras.applications import ResNet50V2
# base_modelV2 = ResNet50V2(include_top=False, weights=None)
# base_modelV2.summary()

In [8]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [6]:
# image preprocessing
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=40,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


batch_size = 32

train_data_dir = "D:/Anil/Dataset/tomato/train"     # directory of training data

test_data_dir = "D:/Anil/Dataset/tomato/val"      # directory of Validation data

training_set = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(256, 256),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory(test_data_dir,
                                            target_size=(256, 256),
                                            batch_size=batch_size,
                                            class_mode='categorical')

print(training_set.class_indices)



Found 15156 images belonging to 11 classes.
Found 1891 images belonging to 11 classes.
{'bacterial_spot': 0, 'early_blight': 1, 'fusarium_wilt': 2, 'healthy': 3, 'late_blight': 4, 'leaf_mold': 5, 'mosaic_virus': 6, 'septoria_leaf_spot': 7, 'spider_mites': 8, 'target_spot': 9, 'yellow_leaf_curl_virus': 10}


In [9]:
# checkpoint
#weightpath = "D:/Anil/saved_model/SACNN/ResNet50_tomato_adam_100_epochs.hdf5"
weightpath = "CBAM-keras/saved_models/ResNet50_tomato_adam_100_epochs.hdf5"
checkpointer = ModelCheckpoint(weightpath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

epochs = 100

start_time = datetime.now()
print("start time: "+str(start_time))

#fitting images to CNN
history = model.fit(training_set,
                         steps_per_epoch=training_set.samples//batch_size,
                         validation_data=test_set,
                         epochs=epochs,
                         validation_steps=test_set.samples//batch_size,
                         callbacks=[checkpointer])#callbacks_list)


# #fitting images to CNN
# history = classifier.fit_generator(training_set,
#                                    steps_per_epoch=training_set.samples//batch_size,
#                                    validation_data=test_set,
#                                    epochs=10,
#                                    validation_steps=test_set.samples//batch_size)

# Stop time
stop_time = datetime.now()
print("stop time: "+str(stop_time))

total_time = stop_time - start_time
print("Total time: "+str(total_time))


start time: 2021-12-20 19:01:46.849819
Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
473/473 [==============================] - ETA: 0s - loss: 1.7087 - accuracy: 0.5434WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001FD2371EA68> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (o

473/473 [==============================] - 136s 286ms/step - loss: 0.0860 - accuracy: 0.9724 - val_loss: 0.0902 - val_accuracy: 0.9698

Epoch 00029: val_loss improved from 0.09749 to 0.09018, saving model to CBAM-keras/saved_models\ResNet50_tomato_adam_100_epochs.hdf5
Epoch 30/100
473/473 [==============================] - 138s 290ms/step - loss: 0.0848 - accuracy: 0.9692 - val_loss: 0.2557 - val_accuracy: 0.9211

Epoch 00030: val_loss did not improve from 0.09018
Epoch 31/100
473/473 [==============================] - 136s 287ms/step - loss: 0.0727 - accuracy: 0.9742 - val_loss: 0.4121 - val_accuracy: 0.8787

Epoch 00031: val_loss did not improve from 0.09018
Epoch 32/100
473/473 [==============================] - 136s 286ms/step - loss: 0.0838 - accuracy: 0.9716 - val_loss: 2.3515 - val_accuracy: 0.6679

Epoch 00032: val_loss did not improve from 0.09018
Epoch 33/100
473/473 [==============================] - 136s 287ms/step - loss: 0.0676 - accuracy: 0.9788 - val_loss: 0.0703 - val_

473/473 [==============================] - 136s 287ms/step - loss: 0.0317 - accuracy: 0.9877 - val_loss: 0.2253 - val_accuracy: 0.9290

Epoch 00069: val_loss did not improve from 0.03890
Epoch 70/100
473/473 [==============================] - 135s 286ms/step - loss: 0.0291 - accuracy: 0.9898 - val_loss: 2.2479 - val_accuracy: 0.6992

Epoch 00070: val_loss did not improve from 0.03890
Epoch 71/100
473/473 [==============================] - 135s 286ms/step - loss: 0.0446 - accuracy: 0.9870 - val_loss: 0.7043 - val_accuracy: 0.8739

Epoch 00071: val_loss did not improve from 0.03890
Epoch 72/100
473/473 [==============================] - 136s 286ms/step - loss: 0.0360 - accuracy: 0.9906 - val_loss: 0.2549 - val_accuracy: 0.9285

Epoch 00072: val_loss did not improve from 0.03890
Epoch 73/100
473/473 [==============================] - 136s 287ms/step - loss: 0.0291 - accuracy: 0.9911 - val_loss: 7.0847 - val_accuracy: 0.4211

Epoch 00073: val_loss did not improve from 0.03890
Epoch 74/100


In [10]:
# another way to save it.
import pandas as pd
hist_df = pd.DataFrame(history.history) 
#hist_csv_file = "D:/Anil/saved_model/SACNN/history_log_lab_ResNet50_adam_80epochs.csv"
hist_csv_file = "CBAM-keras/saved_models/ResNet50_tomato_adam_100_epochs.csv"
# hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w', newline='') as f:
    hist_df.to_csv(f)


In [ ]:
#plotting training values
import matplotlib.pyplot as plt
import seaborn as sns
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training_Acc')
plt.plot(epochs, val_acc, color='blue', label='Validation_Acc')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='best')
plt.savefig("D:/Anil/saved_model/SACNN/accuracy_graph_tain_val_lab_ResNet_adam_80epochs.png", bbox_inches="tight", pad_inches=2)
plt.show()

In [ ]:
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train_loss', 'Val_loss'], loc='best')
plt.legend()

plt.savefig("D:/Anil/saved_model/SACNN/loss_graph_train_val_ResNet50_lab_adam_80epochs.png", bbox_inches="tight", pad_inches=2)
plt.show()


In [ ]:
#loss plot
plt.plot(epochs, acc, color='green', label='Training_Acc')
plt.plot(epochs, loss, color='red', label='Training_Loss')
plt.title('Training Accuracy and Loss')
plt.xlabel('Epochs')
plt.ylabel('ACC&Loss')
plt.legend(['Acc', 'Loss'], loc='best')
plt.legend()

plt.savefig("D:/Anil/saved_model/SACNN/train_accuracy_loss_graph_lab_ResNet50_adam_80epochs.png", bbox_inches="tight", pad_inches=2)
plt.show()


In [ ]:
# testing of model
import os
test_data_dir = 'D:/Anil/Dataset/tomato/test_imgs'
batch_size = 32
img_width, img_height = 224, 224


In [ ]:
def gen_image_label(directory):
    ''' A generator that yields (label, id, jpg_filename) tuple.'''
    for root, dirs, files in os.walk(directory):
        for f in files:
            _, ext = os.path.splitext(f)
            if ext != '.jpg':
                continue
            basename = os.path.basename(f)
            splits = basename.split('.')
            if len(splits) == 3:
                label, id_, ext = splits
            else:
                label = None
                id_, ext = splits
            fullname = os.path.join(root, f)
            yield label, int(id_), fullname
            

In [ ]:
import pandas as pd
# Wrap testing data into pandas' DataFrame.
lst = list(gen_image_label(test_data_dir))
test_df = pd.DataFrame(lst, columns=['label', 'id', 'filename'])
test_df = test_df.sort_values(by=['label', 'id'])
test_df['label_code'] = test_df.label.map({'bacterial_spot': 0, 'early_blight': 1, 'fusarium_wilt': 2, 'healthy': 3, 
                                           'late_blight': 4, 'leaf_mold': 5, 'mosaic_virus': 6, 'septoria_leaf_spot': 7, 
                                           'spider_mites': 8, 'target_spot': 9, 'yellow_leaf_curl_virus': 10})

test_df.head(100)

In [ ]:
Y_true = test_df.label_code
print(Y_true)

In [ ]:
# No need to run again once the file save in computer
import numpy as np
import cv2
img_width = 224
img_height = 224
images = []
dim = (224,224)
for img in test_df.filename:
#     img = image.load_img(img, target_size=(img_width, img_height))
#     img = image.img_to_array(img)
    img = cv2.imread(img)
    img = cv2.resize(img,dim,interpolation=cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    img = img/255.0
    img = np.expand_dims(img, axis=0)
    images.append(img)
# print(images)
# np.save("D:/Anil/Dataset/ageng_conf/test_imgs_299x299", images)
# stack up images list to pass for prediction
images = np.vstack(images)


In [ ]:
model_path = "D:/Anil/saved_model/SACNN/ResNet50_tomato_lab_adam_80_epochs.hdf5"
model = load_model(model_path)

In [ ]:
test_start = datetime.now()
Y_pred = model.predict(images, batch_size=8)
Y_pred_classes = np.argmax(Y_pred, axis = 1)
# classes = model.predict_classes(images, batch_size=8)
print(Y_pred_classes)
test_finish = datetime.now()
test_time = test_finish - test_start
print(test_time)

In [ ]:
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

cls_name = ['0','1','2','3','4','5','6','7','8','9','10']

ytrue = Y_true.values.flatten()
cm = confusion_matrix(ytrue, Y_pred_classes)
fig, ax = plot_confusion_matrix(conf_mat=cm,
                                figsize = (11,9),
                               show_absolute=False,
                               show_normed=True,
                               colorbar=True,
                               class_names = cls_name)
plt.show()
# print(cm)

In [ ]:
fig.savefig("D:/Anil/saved_model/SACNN/confusion_matrix_ResNet50_lab_saved_adam_best_model.png", bbox_inches = 'tight', dpi = 600)


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_true, Y_pred_classes))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_true, Y_pred_classes)